In [1]:
import nagisa
import requests
from bs4 import BeautifulSoup

[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [12]:
source_text = "しかし私は知ってよいことと知ってはならないこととの区別は知っている。"
source_text2 = "まあそんなことはどうでもよい。かりにそうであったとしても、私は決して読みはしない。"

words = nagisa.extract(source_text2, extract_postags=['名詞', '動詞', '形容詞'])


print(words.words)

['こと', 'あっ', 'し', '読み', 'し']


In [6]:
requests.get("https://jisho.org/api/v1/search/words?keyword=知っ").json()

{'meta': {'status': 200},
 'data': [{'slug': '知ったかぶり',
   'is_common': False,
   'tags': [],
   'jlpt': [],
   'japanese': [{'word': '知ったかぶり', 'reading': 'しったかぶり'},
    {'word': '知ったか振り', 'reading': 'しったかぶり'}],
   'senses': [{'english_definitions': ['pretending to know'],
     'parts_of_speech': ['Noun', 'Suru verb'],
     'links': [],
     'tags': [],
     'restrictions': [],
     'see_also': [],
     'antonyms': [],
     'source': [],
     'info': []}],
   'attribution': {'jmdict': True, 'jmnedict': False, 'dbpedia': False}},
  {'slug': '知った事ではない',
   'is_common': False,
   'tags': [],
   'jlpt': [],
   'japanese': [{'word': '知った事ではない', 'reading': 'しったことではない'},
    {'word': '知ったことではない', 'reading': 'しったことではない'}],
   'senses': [{'english_definitions': ['to be of no concern to one',
      'to be nothing to do with one'],
     'parts_of_speech': ['Expressions (phrases, clauses, etc.)',
      'I-adjective (keiyoushi)'],
     'links': [],
     'tags': [],
     'restrictions': [],
     'see